In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [53]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [54]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Lebu,CL,2022-03-26 03:26:06,-37.6167,-73.6500,55.08,92,3,12.44
1,1,Rikitea,PF,2022-03-26 03:23:11,-23.1203,-134.9692,79.95,72,62,16.84
2,2,Clyde River,CA,2022-03-26 03:26:06,70.4692,-68.5914,-7.37,77,100,4.61
3,3,Hermanus,ZA,2022-03-26 03:26:06,-34.4187,19.2345,60.80,83,56,13.65
4,4,Sola,VU,2022-03-26 03:26:07,-13.8833,167.5500,81.27,89,75,0.00


In [55]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [56]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [57]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
#heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [58]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds= city_data_df["Cloudiness"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [64]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \(city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Rikitea,PF,2022-03-26 03:23:11,-23.1203,-134.9692,79.95,72,62,16.84
4,4,Sola,VU,2022-03-26 03:26:07,-13.8833,167.5500,81.27,89,75,0.00
5,5,Arraial Do Cabo,BR,2022-03-26 03:26:07,-22.9661,-42.0278,76.80,91,79,23.09
9,9,Kidal,ML,2022-03-26 03:26:09,18.4411,1.4078,81.79,15,73,12.68
15,15,Vaini,TO,2022-03-26 03:26:11,-21.2000,-175.2000,82.56,74,20,11.50
...,...,...,...,...,...,...,...,...,...,...
557,557,Launceston,AU,2022-03-26 03:33:48,-41.4500,147.1667,75.61,50,5,10.36
558,558,Champerico,GT,2022-03-26 03:34:18,14.3000,-91.9167,82.11,69,98,0.89
559,559,Pemangkat,ID,2022-03-26 03:34:19,1.1667,108.9667,84.70,69,94,5.50
561,561,Townsville,AU,2022-03-26 03:33:49,-19.2500,146.8000,85.08,62,100,17.27


In [65]:
preferred_cities_df.count()

City_ID       156
City          156
Country       156
Date          156
Lat           156
Lng           156
Max Temp      156
Humidity      156
Cloudiness    156
Wind Speed    156
dtype: int64

In [66]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.95,-23.1203,-134.9692,
4,Sola,VU,81.27,-13.8833,167.5500,
5,Arraial Do Cabo,BR,76.80,-22.9661,-42.0278,
9,Kidal,ML,81.79,18.4411,1.4078,
15,Vaini,TO,82.56,-21.2000,-175.2000,
17,Carnarvon,AU,84.27,-24.8667,113.6333,
18,Kita,ML,81.99,13.0349,-9.4895,
20,Nampula,MZ,75.15,-15.1165,39.2666,
22,Isangel,VU,86.11,-19.5500,169.2667,
24,Atuona,PF,79.23,-9.8000,-139.0333,


In [67]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [68]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [69]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEAWmwyhIyPMl8PaZeFBZGol8a2uo6FxtEllHtwT92kNcre9tbMk8wHmiDcglk-AswmTR_NodLozi_hMZfuOWFs-vpqKc9soD1Bf2fPzTdUF2sT3-l4BFr5rrVZFoXWt9-Yk916uPyvdfTFz_uWMQfVUuUuArQrWYcuBH1AM9A9NWOecWI975b2xp27iBuSUxi0i-H_rn5OG_Zww9FuXsDoQFMjwyZUB9wL5OEDvVFTMx7g0J06CHMOuq1BQGhPWW96XHKIUJ_t32Aptcu7azXcDJFOA582JeIVZWGYwbkOnw-6slnFXtwhqCXq16mWHfKGhanJTFLmR9Iu8hDCSA4UjC9WHUkj06KlAr9SF6c6Xh2QKaklkPzFkh6I4stUNnvnVCkVBMNvIHWqULsZz2h9bnOYWkxZZoSrnAVNT8RyYkG6Y9-ACgWgE',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [70]:
len(hotels["results"])

20

In [71]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [72]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


KeyboardInterrupt: 

In [73]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.95,-23.1203,-134.9692,People ThankYou
4,Sola,VU,81.27,-13.8833,167.5500,Leumerus Bungalows
5,Arraial Do Cabo,BR,76.80,-22.9661,-42.0278,Pousada Porto Praia
9,Kidal,ML,81.79,18.4411,1.4078,
15,Vaini,TO,82.56,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...
557,Launceston,AU,75.61,-41.4500,147.1667,
558,Champerico,GT,82.11,14.3000,-91.9167,
559,Pemangkat,ID,84.70,1.1667,108.9667,
561,Townsville,AU,85.08,-19.2500,146.8000,


In [74]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [75]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [76]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [77]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [78]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [79]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))